In [124]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
import sqlite3
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import tool as langchain_tool
import json
from src.constants import schema_dict
from src.prompts import FINANCIAL_ANALYST_AGENT_PROMPT
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.memory import ConversationBufferMemory

In [58]:
@langchain_tool
def sql_query_executor_tool(query: str) -> str:
    """This function connects to the database, executes query and returns result
    
    Parameters:
    query (str): SQL query to be executed

    Returns:
    str: Resulf of SQL query in JSON format
    """

    print(f"\n--- 🛠️ Tool Called: search_information with query: '{query}' ---")

    conn = sqlite3.connect(os.environ["DB_NAME"])
    conn.row_factory = sqlite3.Row
    cur = conn.cursor()
    
    result = cur.execute(query)
    column_names = [desc[0] for desc in result.description]
    # print(f"\n\nColumn names: {column_names}\n\n")
    rows = result.fetchall()
    # print(f"\n\nRows: {rows}\n\n")
    final_data_rows = [dict(zip(column_names, row)) for row in rows]
    print(final_data_rows)
    conn.close()
    return final_data_rows

In [59]:
try:
    llm = ChatOpenAI(model=os.environ["LLM_MODEL"], temperature=0)
except Exception as e:
    print(f"\n\nError initializing ChatOpenAI: \n{e}")

In [ ]:

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", FINANCIAL_ANALYST_AGENT_PROMPT),
    ("human", "{user_query}"),
    ("placeholder", "{agent_scratchpad}")
])

In [125]:
tools = [sql_query_executor_tool]
# Create the agent, binding the LLM, tools, and prompt together.
agent = create_tool_calling_agent(
    llm=llm,
    tools=[sql_query_executor_tool],
    prompt=agent_prompt,
)

class AgentMemory(ConversationBufferMemory):
    """Custom memory class to handle conversation history."""
    def load_memory_variables(self, inputs):
        """Load memory variables, ensuring chat history is included."""
        memory_variables = super().load_memory_variables(inputs)
        return memory_variables
    
    def save_context(self, inputs: dict, outputs: dict) -> None:
        # Intercept inputs/outputs before saving
        print("🔍 Intercepted Inputs:", inputs)
        print("🔍 Intercepted Outputs:", outputs)
        
        # Example: Drop tool calls from memory
        clean_outputs = {"output": outputs["output"].replace("[TOOL_CALL]", "")}
        
        # Call parent with modified values
        super().save_context(inputs, clean_outputs)


memory = AgentMemory(memory_key="chat_history", return_messages=True)
#  AgentExecutor is the runtime that invokes the agent and executes the chosen tools.
#  The 'tools' argument is not needed here as they are already bound to the agent.
agent_executor = AgentExecutor(agent=agent, verbose=True, tools=tools, max_iterations=1, 
                               return_intermediate_steps=True, memory=memory)

In [126]:
response = agent_executor.invoke({
    "user_query": "Give me the summary of the all the tables, range data points, number of records etc.",
    "db_schema": json.dumps(schema_dict, indent=2)
})



> Entering new AgentExecutor chain...

Invoking: `sql_query_executor_tool` with `{'query': "SELECT 'cities' AS table_name, COUNT(*) AS row_count FROM cities;"}`



--- 🛠️ Tool Called: search_information with query: 'SELECT 'cities' AS table_name, COUNT(*) AS row_count FROM cities;' ---
[{'table_name': 'cities', 'row_count': 96}]
[{'table_name': 'cities', 'row_count': 96}]
Invoking: `sql_query_executor_tool` with `{'query': "SELECT 'countries' AS table_name, COUNT(*) AS row_count FROM countries;"}`



--- 🛠️ Tool Called: search_information with query: 'SELECT 'countries' AS table_name, COUNT(*) AS row_count FROM countries;' ---
[{'table_name': 'countries', 'row_count': 206}]
[{'table_name': 'countries', 'row_count': 206}]
Invoking: `sql_query_executor_tool` with `{'query': "SELECT 'customers' AS table_name, COUNT(*) AS row_count FROM customers;"}`



--- 🛠️ Tool Called: search_information with query: 'SELECT 'customers' AS table_name, COUNT(*) AS row_count FROM customers;' ---
[{'tabl

ValueError: One input key expected got ['db_schema', 'user_query']

In [ ]:
print(response["intermediate_steps"][0])
print(type(response["intermediate_steps"][0]), len(response["intermediate_steps"][0]), len(response["intermediate_steps"][0]))
print(type(response["intermediate_steps"][0][0]))
print(type(response["intermediate_steps"][0][0].tool), type(response["intermediate_steps"][0][0].tool_input), type(response["intermediate_steps"][0][0].log))
print(f'\n\nTool: \n{response["intermediate_steps"][0][0].tool}')
# print(response["intermediate_steps"][0][0].tool_input)
print(f"\n\nTool input: \n{response['intermediate_steps'][0][0].tool_input}")
# print(response["intermediate_steps"][0][0].log)
print(f"\n\nLog: \n{response['intermediate_steps'][0][0].log}")

(ToolAgentAction(tool='sql_query_executor_tool', tool_input={'query': "SELECT 'cities' AS table_name, COUNT(*) AS record_count, MIN(CityID) AS min_CityID, MAX(CityID) AS max_CityID, MIN(Zipcode) AS min_Zipcode, MAX(Zipcode) AS max_Zipcode, MIN(CountryID) AS min_CountryID, MAX(CountryID) AS max_CountryID FROM cities;"}, log='\nInvoking: `sql_query_executor_tool` with `{\'query\': "SELECT \'cities\' AS table_name, COUNT(*) AS record_count, MIN(CityID) AS min_CityID, MAX(CityID) AS max_CityID, MIN(Zipcode) AS min_Zipcode, MAX(Zipcode) AS max_Zipcode, MIN(CountryID) AS min_CountryID, MAX(CountryID) AS max_CountryID FROM cities;"}`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_0WP53DzbeTKgIO8voaFImseu', 'function': {'arguments': '{"query": "SELECT \'cities\' AS table_name, COUNT(*) AS record_count, MIN(CityID) AS min_CityID, MAX(CityID) AS max_CityID, MIN(Zipcode) AS min_Zipcode, MAX(Zipcode) AS max_Zipcode, MIN(CountryID) AS min_

TypeError: object of type 'ToolAgentAction' has no len()

In [121]:
# len(response["intermediate_steps"][0])
print(response["intermediate_steps"][0][1])

[{'table_name': 'cities', 'record_count': 96, 'min_CityID': 1, 'max_CityID': 96, 'min_Zipcode': 157, 'max_Zipcode': 97859, 'min_CountryID': 32, 'max_CountryID': 32}]


In [ ]:
# async def run_agent_with_tool(query: str):
#   """Invokes the agent executor with a query and prints the final response."""
#   print(f"\n--- 🏃 Running Agent with Query: '{query}' ---")
#   try:
#       response = await agent_executor.ainvoke({"user_query": query})
#       print("\n--- ✅ Final Agent Response ---")
#       print(response["output"])
#   except Exception as e:
#       print(f"\n🛑 An error occurred during agent execution: {e}")


In [ ]:
# import os, getpass
# import asyncio
# import nest_asyncio
# from typing import List
# from dotenv import load_dotenv
# import logging

# async def main():
#     """Runs all agent queries concurrently."""
#     tasks = [
#         run_agent_with_tool("What is the capital of France?"),
#         run_agent_with_tool("What's the weather like in London?"),
#         run_agent_with_tool("Tell me something about dogs.") # Should trigger the default tool response
#     ]
#     await asyncio.gather(*tasks)

# nest_asyncio.apply()
# asyncio.run(main())



--- 🏃 Running Agent with Query: 'What is the capital of France?' ---

--- 🏃 Running Agent with Query: 'What's the weather like in London?' ---

--- 🏃 Running Agent with Query: 'Tell me something about dogs.' ---


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...

🛑 An error occurred during agent execution: 'Input to ChatPromptTemplate is missing variables {\'\\n    "tables"\'}.  Expected: [\'\\n    "tables"\', \'user_query\'] Received: [\'user_query\', \'intermediate_steps\', \'agent_scratchpad\']\nNote: if you intended {\n    "tables"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{\n    "tables"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '

🛑 An error occurred during agent execution: 'Input to ChatPromptTemplate is missing variables {\'\\n    "tables"\'}.  Expected: [\'\\n    "tables"\', \'user_que